# Hands-On 4: Matrix Computation with MPI

Welcome to Hands-on _Matrix Computation with MPI_. This Hands-on comprises 1 session. Next table shows the document and file needed to develop each one of the exercises.

|  Sessions     | Codes               | files              | 
| --------------| --------------------| ------------------ |
| Session 1     |  LU Factorization   |   lu.c   | 

# `LU Factorization`

This last practice focuses on the parallel implementation of a more advanced numerical problem using the MPI environment that implements the MPI (Message Passing Interface) message passing standard. The objective of the practice is to face a more significant problem with more room for
improvement. This practice consists of $2$ objectives:

1.  Study of the problem and implementation of the parallel version.

2.  Improvement of the cyclical parallel version and experimental
    measurement of performance.

The practice will start from the sequential and parallel version by blocks of rows found in the cloud, and will have to be modified to include a cyclical distribution by rows.

## Description of the problem

A system of equations $Ax=b$ has a unique solution (compatible and determined) when the coefficient matrix has a nonzero determinant. Several direct and iterative techniques for solving equations highlight the LU factorization among the natural methods.

The LU factorization consists of obtaining a pair of matrices, a lower triangular unit (all elements above the main diagonal to zero and the main diagonal itself to one) and another upper triangular (all elements above the diagonal zero and no restrictions on the main diagonal), both
of the same dimension as the starting matrix.

In this way, the resolution of the system of equations is equivalent to the solution of two triangular systems, the programming of which is well known. 

<br>

$$\left.\begin{array}{c}A=LU\\Ax=b\end{array}\right\}
\quad\longrightarrow\quad
LUx=b
\quad\longrightarrow\quad
\left\{\begin{array}{c}Ly=b\\Ux=y\end{array}\right..$$

The calculation of the LU decomposition can be performed by Gaussian
factorization, using the diagonal elements of the matrix as pivots and
making zeros below the diagonal in each column.

The triangular systems can be solved by applying progressive elimination
algorithms (for the case of the lower unit triangular matrix) back
substitution (for the case of the upper triangular matrix).

## Sequential Implementation

In [ ]:
%%writefile lu.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

/* generates a A*X=B problem in which all value for X are 1 */
void generate_ab_problem(int size, double **a_matrix, double *b_array)
{
    int r, c;
    for (r = 0; r < size; r++)
    {
        int sum = 0;
        for (c = 0; c < size; c++)
        {
            int coefficient = -5 + (rand() % 11);
            sum += coefficient;
            a_matrix[r][c] = coefficient;
        }
        b_array[r] = sum;
    }
}

double **alloc_matrix(int size)
{
    double **matrix = (double **)calloc(size, sizeof(double *));
    double *matrix_values = (double *)calloc(size * size, sizeof(double));
    int i;

    for (i = 0; i < size; i++)
        matrix[i] = (matrix_values + size * i);

    return matrix;
}

double *alloc_array(int size)
{
    return (double *)calloc(size, sizeof(double));
}

void print_matrix(double **matrix, int size)
{
    int r, c;
    for (r = 0; r < size; r++)
    {
        for (c = 0; c < size; c++)
            printf("%.2f\t", matrix[r][c]);
        printf("\n");
    }
}

void print_array(double *array, int size)
{
    int i;
    for (i = 0; i < size; i++)
        printf("%.2f\t", array[i]);
    printf("\n");
}

/* performs LU decomposition in place */
void lu_decompose(double **matrix, int size)
{
    int r, c;

    for (c = 0; c < size; c++)
    {
        for (r = c + 1; r < size; r++)
        {
            double factor = matrix[r][c] / matrix[c][c];
            matrix[r][c] = factor;

            int cc;
            for (cc = c + 1; cc < size; cc++)
                matrix[r][cc] -= matrix[c][cc] * factor;
        }
    }
}

/* solves L*Y=B for Y using progressive elimination */
void solve_lyb(double **l_matrix, int size, double *y_array, double *b_array)
{
    int r, c;

    y_array[0] = b_array[0];
    for (r = 1; r < size; r++)
    {
        y_array[r] = b_array[r];
        for (c = 0; c < r; c++)
            y_array[r] -= l_matrix[r][c] * y_array[c];
    }
}

/* solves the U*X=Y for X using regressive substitution */
void solve_uxy(double **u_matrix, int size, double *x_array, double *y_array)
{
    int r, c;

    x_array[size - 1] = y_array[size - 1] / u_matrix[size - 1][size - 1];
    for (r = size - 2; r >= 0; r--)
    {
        for (c = size - 1; c > r; c--)
            x_array[r] -= u_matrix[r][c] * x_array[c];

        x_array[r] += y_array[r];
        x_array[r] /= u_matrix[r][r];
    }
}

int main(int argc, char **argv)
{
    int size = atoi(argv[1]);

    double **A = alloc_matrix(size);
    double *B = alloc_array(size);
    double *Y = alloc_array(size);
    double *X = alloc_array(size);

    generate_ab_problem(size, A, B);

    printf("\nA\n");
    print_matrix(A, size);
    printf("\nB\n");
    print_array(B, size);

    lu_decompose(A, size); // A now contains both L and U
    solve_lyb(A, size, Y, B);
    solve_uxy(A, size, X, Y);

    printf("\nL\\U\n");
    print_matrix(A, size);
    printf("\nY\n");
    print_array(Y, size);
    printf("\nX\n");
    print_array(X, size);
    
    return 0;
}

### Run the Code

In [ ]:
!gcc lu.c -o lu

In [ ]:
!./lu 8

## Work to be done

The behaviour should be analysed and properly justified with the comparison of cyclical parallel version and experimental measurement of performance. The solution must be prepared and delivered containing a description of the implementation, performance graphs (execution time,
speedup, etc.) and the conclusions.

## References

M. Boratto. Hands-On Supercomputing with Parallel Computing. Available: https://github.com/muriloboratto/Hands-On-Supercomputing-with-Parallel-Computing. 2022.

B. Chapman, G. Jost and R. Pas. Using OpenMP: Portable Shared Memory Parallel Programming. The MIT Press, 2007, USA.

F. Almeida, D. Giménez, J. M. Mantas and A. M. Vidal. *Introducción a la
Programación Paralela*. Ed. Paraninfo, 2008, Spain.

Forum, Message Passing Interface. *MPI: A Message-Passing Interface
Standard*. University of Tennessee, 1994, USA.